In [1]:
import numpy as np

In [2]:
pi = np.pi

## Algorithm 1

Given an $N \times N$ matrix $A$ denoted as $\{ a \} _{i,j = 1}^{N,N}$ the routine replaces it by the LU decomposition of a row-wise permutation of itself. $"a"$ and $"N"$ are input. $"a"$ is also output, modified to apply the LU decomposition; $\{ indx_i \} _{i,j = 1}^N$ is an output vector that records the row permutation affected by the partial pivoting; $"d"$ is output and adopts $\pm 1 $ depending on whether the number of row interchanges was even or off. This routine is used in combination with algorithm 2 to solve linear equations or invert a matrix.

In [3]:
def ludecmp(A, N):

    #this function performs LU decomposition of a matrix A

    tiny = 1e-20

    d = 1.  
    # initialize vectors
    vv = np.zeros(N)
    indx = [0] * N

    for ii in range(0,N): # python index starts at i = 0, range does not include N

        a_max = 0. # big = 0


        # search for largest entry in row and store
        for jj in range(0,N):  

            a_ij = np.abs(A[ii,jj])  #temp = abs(a_ij)

            if (a_ij > a_max):
                a_max = a_ij


        # check if matrix is singular; exit if singular    
        if (a_max == 0.):
                print("This matrix is singular")
                exit

        # store value to divide row by
        vv[ii] = 1./a_max
    #print(vv)

    for jj in range(0,N):

        for ii in range(0,jj):

            sums = A[ii,jj]

            for kk in range(0,ii):

                sums = sums - A[ii,kk] * A[kk, jj]
            
            A[ii,jj] = sums

        a_max = 0.

        for ii in range(jj,N): 
            sums = A[ii,jj]

            for kk in range(0,jj):

                sums = sums - A[ii,kk] * A[kk, jj]

            A[ii,jj] = sums

            dummy = vv[ii] * np.abs(sums)

        # storing index for pivot
            if (dummy >= a_max):
                a_max = dummy

                i_max = ii
               
        # pivoting
        if (jj != i_max):

            for kk in range(0,N):

                dummy = A[i_max,kk]

                A[i_max,kk] = A[jj,kk]

                A[jj,kk] = dummy

        #storing account of pivot
            d = -d

            vv[i_max] = vv[jj]

        indx[jj] = i_max
 
        # account for division by zero

        if (A[jj,jj] == 0.):
            A[jj,jj] = tiny
        

        # for jj not the last value: jj = 0, 1 
        if (jj != N-1): 
            dummy = 1/A[jj,jj]

            for ii in range(jj+1, N): 
                A[ii,jj] = A[ii,jj] * dummy

    
    #print(A)
    #print(indx)
    return indx, d, A


## Algorithm 2

Solve the set of $N$ linear equations $\textbf{A} \vec{x} = \vec{b}$. Matrix $\{ a \} _{i,j = 1}^{N,N}$ is the LU decomposition of original matrix $A$ obtained from Algorithm 1. Vector $\{ indx_i \} _{i,j = 1}^N$ is input as the permutation vector returned by Algorithm 1. Vector $\{b_i\} _{i=1}^N$ is input as the right hand side vector $\vec{b}$ but returns with solution vector $\vec{x}$. Inputs $\{ a \} _{i,j = 1}^{N,N}$, $N$ and $\{ indx_i \} _{i,j = 1}^N$ 

In [4]:
def lubksb(b,A,N,indx):
    ii = -1 #python index starts at 0, so we go one less than 0
    for i in range(0,N):
        ll = indx[i]
        #print(ll)
        sums = b[ll]
        #print(i,sums)
        b[ll] = b[i]

        if (ii != -1):
            for jj in range(ii, i):
                sums -= A[i,jj] * b[jj]
                #print(jj,sums)
        elif (sums != 0.):
            ii = i
            

        b[i] = sums
        #print(i,b[i])

    for i in range(N-1,-1,-1):
        sums = b[i]
        for jj in range(i+1,N):
            sums -=  A[i,jj] * b[jj]

        b[i] = sums/A[i,i]
        #print(i,b[i])

    return b

## Delivering results

### Case 1

$$\textbf{A} = \left[ \begin{array}{ccc} 1 & 2 & -1 \\ 6 & -5 & 4 \\ -9 & 8 &-7 \end{array}\right] ; \textbf{b} = \left[ \begin{array}{c} 2\pi \\ 5\pi \\ -8\pi\end{array}\right]$$

In [5]:
# construct A matrix
A = np.array([[1., 2., -1.], [6., -5., 4.] , [-9., 8., -7.]])
b = np.array([2.*pi, 5.*pi,-8.*pi])
# Store size N for any N x N matrix

N = np.size(b)

indx, d, A_new = ludecmp(A,N)

x = lubksb(b,A_new,N,indx)

print(x)

[3.14159265 3.14159265 3.14159265]


### Case 2
$$\textbf{A} = \left[ \begin{array}{ccc} \pi & 3\pi & 2\pi \\ 0 & 1 & -2/3 \\ -\pi & -3\pi &2\pi \end{array}\right] ; \textbf{b} = \left[ \begin{array}{c} 3 \\ 0 \\ -1\end{array}\right]$$

In [6]:
# construct A matrix
A = np.array([[pi, 3.*pi, 2*pi], [0., 1., -2./3.] , [-pi, -3.*pi, 2.*pi]])
b = np.array([3., 0.,-1.])
# Store size N for any N x N matrix

N = np.size(b)

indx, d, A_new = ludecmp(A,N)

x = lubksb(b,A_new,N,indx)

print(x)

[0.31830989 0.1061033  0.15915494]


## Verify our Results
We can check our algorithms by using scipy's LU decomposition

In [7]:
from scipy.linalg import lu, lu_factor, lu_solve

In [8]:
A =np.array([[1, 2, -1], [6, -5, 4] , [-9, 8, -7]])
b = np.array([2*pi, 5*pi,-8*pi])
b.shape=(3,1)

P,L,U = lu(A)

LU,p = lu_factor(A)

lu_solve((LU,p),b)

array([[3.14159265],
       [3.14159265],
       [3.14159265]])

In [9]:
A = np.array([[pi, 3.*pi, 2*pi], [0., 1., -2./3.] , [-pi, -3.*pi, 2.*pi]])
b = np.array([3., 0.,-1.])
b.shape=(3,1)

P,L,U = lu(A)

LU,p = lu_factor(A)

lu_solve((LU,p),b)

array([[0.31830989],
       [0.1061033 ],
       [0.15915494]])